In [32]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_score, log_loss
from sklearn.linear_model import LogisticRegression

In [22]:
# Load dataset

X_train = pd.read_csv('train_features.csv', index_col=0)
Y_train = pd.read_csv('train_targets_scored.csv', index_col=0)
X_test = pd.read_csv('test_features.csv', index_col=0)

In [75]:
# Load submission sample
sample = pd.read_csv('sample_submission.csv', index_col=0)

# Print the shape of the sample
sample.shape

(3982, 206)

In [23]:
# Print the shape of X_train
print(X_train.shape)

# Quickviw of X_train
X_train.head()

(23814, 875)


,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [24]:
# Print the shape of Y_train
print(Y_train.shape)

# Quickviw of Y_train
Y_train.head()

(23814, 206)


,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000779bfc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000a6266a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_0015fd391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_001626bd3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Print the shape of X_test
print(X_test.shape)

# Quickviw of X_test
X_test.head()

(3982, 875)


,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,-0.2140,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,-1.8820,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,-0.3390,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,0.5998,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,0.5128,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


### Data Preparation for MVP

In [27]:
# Create a mask for treated samples for train
mask = (X_train.cp_type != 'ctl_vehicle')
mask.shape

(23814,)

In [28]:
# Remove controls in X_train
X_train = X_train[mask]

# Remove controls in Y_train
Y_train = Y_train[mask]

# Print the shape of X and Y train
print(X_train.shape)
print(Y_train.shape)

(21948, 875)
(21948, 206)


In [29]:
# Create a mask for treated samples for train
mask = (X_test.cp_type != 'ctl_vehicle')
mask.shape

(3982,)

In [30]:
# Remove controls in X_test
X_test = X_test[mask]

# Print the shape of X and Y test
print(X_test.shape)

(3624, 875)
(3624, 206)


In [76]:
# Remove controls in sample
sample = sample[mask]

# Print the shape of the submission sample
sample.shape

(3624, 206)

In [31]:
# Remove the condition columns in X_train and _test

remove_cols = ['cp_type', 'cp_time', 'cp_dose']

# Remove controls in X_train
X_train.drop(columns=remove_cols, inplace=True)

# Remove controls in X_test
X_test.drop(columns=remove_cols, inplace=True)

# Print the shape of X and Y train
print(X_train.shape)
print(X_test.shape)

(21948, 872)
(3624, 872)


### Baseline: independent models for the 206 labels

In [39]:
# Convert all the dataset to np.assy

X_train = X_train.values
Y_train = Y_train.values
X_test = X_test.values

In [41]:
# Print out the dtypes of X_ and Y_

type(X_train), type(Y_train), type(X_test)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [46]:
# Scale the dataset
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

In [47]:
# Scale the dataset
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
# Fit an independent logistic regression model for each class using the OneVsRestClassifier wrapper.

base_lr = LogisticRegression(max_iter=10000) # Create a lr object and change the max_iter to 10000

ovr = OneVsRestClassifier(base_lr) # Create a ovr object using lr

In [53]:
# fit on train
ovr.fit(X_train_scaled, Y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=10000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [79]:
# Predict Y
Y_pred_ovr = ovr.predict(X_test_scaled)

In [80]:
# Convert predicted Y in a submission format

df_sub = pd.DataFrame(Y_pred_ovr, 
                      index=sample.index, 
                      columns = sample.columns) # Don't remove the controls

# Print the shape of the df_sub
print(df_sub.shape)

# Quick view of df_sub
df_sub.head()

(3624, 206)


,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_001897cda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_00276f245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_0027f1083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_006fc47b8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# Compute the counts for each drug target

df_sub.sum(axis=0).sort_values(ascending=False)

nfkb_inhibitor                           161
proteasome_inhibitor                     142
egfr_inhibitor                            44
tubulin_inhibitor                         44
adrenergic_receptor_agonist               41
                                        ... 
lxr_agonist                                0
mdm_inhibitor                              0
mek_inhibitor                              0
mineralocorticoid_receptor_antagonist      0
5-alpha_reductase_inhibitor                0
Length: 206, dtype: int64